In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import pandas as pd
import numpy as np

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Fit_pulse_anomaly/health_fitness_tracking_365days.csv')

In [ ]:
df.columns = df.columns.str.lower().str.strip()

print("Original Columns:", df.columns.tolist())
df.head()

Original Columns: ['user_id', 'age', 'gender', 'date', 'steps', 'heart_rate_avg', 'sleep_hours', 'calories_burned', 'exercise_minutes', 'stress_level', 'weight_kg', 'bmi']


,user_id,age,gender,date,steps,heart_rate_avg,sleep_hours,calories_burned,exercise_minutes,stress_level,weight_kg,bmi
0,0,56,F,2024-09-06,9341,62.029621,9.368819,2230.230419,0.623979,2,73.496429,22.471978
1,0,56,F,2024-09-07,10873,59.062818,6.358311,1840.454777,109.208987,3,68.237867,22.569858
2,0,56,F,2024-09-08,6638,58.494078,6.099619,2284.231946,3.083319,4,81.687890,17.595609
3,0,56,F,2024-09-09,6062,56.546095,7.584023,1620.464266,22.023327,10,86.379884,20.154137
4,0,56,F,2024-09-10,10399,59.507172,7.327957,2264.528312,76.483061,8,81.782982,32.624040


In [ ]:
df.rename(columns={
    'date_time': 'timestamp',
    'date': 'timestamp',
    'heart_rate_avg': 'heart_rate',
    'step_count': 'steps',
    'total_steps': 'steps',
    'sleep_hours': 'sleep'
}, inplace=True)

In [ ]:
df = df[['timestamp', 'heart_rate', 'steps','sleep']].copy()
df.head()

,timestamp,heart_rate,steps,sleep
0,2024-09-06,62.029621,9341,9.368819
1,2024-09-07,59.062818,10873,6.358311
2,2024-09-08,58.494078,6638,6.099619
3,2024-09-09,56.546095,6062,7.584023
4,2024-09-10,59.507172,10399,7.327957


In [ ]:
df.rename(columns={'date': 'timestamp'}, inplace=True)
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
# The timestamp column is already timezone-aware (UTC), so tz_localize is not needed and causes an error.
# df['timestamp'] = df['timestamp'].dt.tz_localize('UTC', nonexistent='shift_forward', ambiguous='NaT')

In [ ]:
df[['heart_rate', 'steps','sleep']] = df[['heart_rate', 'steps','sleep']].interpolate(method='linear')
df.fillna(method='ffill', inplace=True)


/tmp/ipython-input-4239613350.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


In [ ]:
df.set_index('timestamp', inplace=True)

minute_df = pd.DataFrame({
    'heart_rate': df['heart_rate'].resample('1T').mean(),
    'steps': df['steps'].resample('1T').mean(),
    'sleep': df['sleep'].resample('1T').max()
}).reset_index()


/tmp/ipython-input-1610516864.py:4: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  'heart_rate': df['heart_rate'].resample('1T').mean(),
/tmp/ipython-input-1610516864.py:5: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  'steps': df['steps'].resample('1T').mean(),
/tmp/ipython-input-1610516864.py:6: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  'sleep': df['sleep'].resample('1T').max()


In [ ]:

minute_df = minute_df[['timestamp', 'heart_rate', 'steps', 'sleep']]
minute_df.head()


,timestamp,heart_rate,steps,sleep
0,2024-09-06 00:00:00,64.484483,8490.407,11.172378
1,2024-09-06 00:01:00,NaN,NaN,NaN
2,2024-09-06 00:02:00,NaN,NaN,NaN
3,2024-09-06 00:03:00,NaN,NaN,NaN
4,2024-09-06 00:04:00,NaN,NaN,NaN


In [ ]:
from google.colab import files

minute_df.to_csv("clean_fitness_data_minute.csv", index=False)
files.download("clean_fitness_data_minute.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df.head()

,heart_rate,steps,sleep
timestamp,,,
2024-09-06,62.029621,9341,9.368819
2024-09-07,59.062818,10873,6.358311
2024-09-08,58.494078,6638,6.099619
2024-09-09,56.546095,6062,7.584023
2024-09-10,59.507172,10399,7.327957
